# Zero-Shot Weed Detection with LLAMA VLMs (Batch Runner)

In [ ]:
# --- Cell 1: Zero-shot Weed Detection with Meta LLaMA Vision (Together AI) ---

import os
import base64
import time
from pathlib import Path

import pandas as pd
from tqdm import tqdm
from together import Together

# ==== REQUIRED: Set your Together API Key ====
TOGETHER_API_KEY = ""
client = Together(api_key=TOGETHER_API_KEY)

# ==== User Inputs ====
IMAGE_FOLDER = "Joint_Val"
OUTPUT_EXCEL = "llama4_Scout_weed_results.xlsx"
OUTPUT_CSV   = "llama4_Scout_weed_results.csv"

# Choose the LLaMA Vision model from Together AI
MODEL = "meta-llama/Llama-4-Scout-17B-16E-Instruct"
# Alternative examples:
# meta-llama/Llama-4-Scout-17B-16E-Instruct
# meta-llama/Llama-4-Maverick-17B-128E-Instruct-FP8

# ==================== PROMPT ====================
PROMPT = (
    "Analyze the image and provide output in exactly this format, and nothing extra:\n"
    "Weed Detection: <Yes or No>\n"
    "Weed Location: <Mention Position in the Image>\n"
    "Reasoning: <Explain Reasoning>\n"
    "Crop Growth: <Early, Growing or Full Grown>\n"
    "Crop Type: <Predict Crop Type>\n"
    "If your main response is 'No Weed', Only then rewrite the response again with V2_ at the start of each field, assuming weed is present.\n"
)

# Optional: limit to certain image extensions
IMAGE_EXTS = {".jpg", ".jpeg", ".png", ".bmp", ".tif", ".tiff", ".webp"}

# Retry/backoff settings
MAX_RETRIES = 5
INITIAL_BACKOFF = 2.0  # seconds

# --- Helper: encode + call Together API ---
def call_together_vision(image_path: Path, prompt: str, model: str) -> str:
    """Call Together AI API with an image + prompt, with retries."""
    try:
        with open(image_path, "rb") as image_file:
            encoded = base64.b64encode(image_file.read()).decode("utf-8")
        base64_url = f"data:image/jpeg;base64,{encoded}"
    except Exception as e:
        return f"ERROR: Failed to encode image - {e}"

    backoff = INITIAL_BACKOFF
    for attempt in range(1, MAX_RETRIES + 1):
        try:
            response = client.chat.completions.create(
                model=model,
                messages=[
                    {
                        "role": "user",
                        "content": [
                            {"type": "text", "text": prompt},
                            {"type": "image_url", "image_url": {"url": base64_url}},
                        ],
                    }
                ],
            )
            text = response.choices[0].message.content
            return text.strip() if text else ""
        except Exception as e:
            if attempt == MAX_RETRIES:
                return f"ERROR: Max retries exceeded - {e}"
            time.sleep(backoff)
            backoff *= 2
    return "ERROR: Unknown error occurred."

# --- Run Batch Inference ---
img_dir = Path(IMAGE_FOLDER)
assert img_dir.is_dir(), f"IMAGE_FOLDER does not exist: {img_dir}"

images = sorted([p for p in img_dir.iterdir() if p.suffix.lower() in IMAGE_EXTS])
print(f"Found {len(images)} images to process.")

out_xlsx = Path(OUTPUT_EXCEL)
out_csv = Path(OUTPUT_CSV)
out_xlsx.parent.mkdir(parents=True, exist_ok=True)
out_csv.parent.mkdir(parents=True, exist_ok=True)

rows = []
for i, p in enumerate(tqdm(images, desc="Processing images"), start=1):
    try:
        llama_response = call_together_vision(p, PROMPT, MODEL)
    except Exception as e:
        llama_response = f"ERROR: {e}"

    row = {"Image": p.name, "Llama_Response": llama_response}
    rows.append(row)

    # Save incrementally for safety
    df = pd.DataFrame(rows, columns=["Image", "Llama_Response"])
    df.to_excel(out_xlsx, index=False)
    df.to_csv(out_csv, index=False, encoding="utf-8")

    if i % 500 == 0 or i == len(images):
        print(f"Saved progress at {i}/{len(images)} images")

print(f"\nFinal results saved to:\n Excel: {out_xlsx}\n CSV:   {out_csv}")


Found 441 images to process.


Processing images: 100%|██████████| 441/441 [14:14<00:00,  1.94s/it]

Saved progress at 441/441 images

Final results saved to:
 Excel: D:\Khalifa University 2024\Conferences - Research\Al-Ain 2025\llama4_Scout_weed_results.xlsx
 CSV:   D:\Khalifa University 2024\Conferences - Research\Al-Ain 2025\llama4_Scout_weed_results.csv
